In [111]:
# import lib
import fiftyone.zoo as foz
import fiftyone as fo
from fiftyone import ViewField as F
from keras.applications import VGG19
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

In [147]:
# define samples count
subset_samples = 250

In [224]:
# List all available datasets
datasets = fo.list_datasets()
# Delete 'person_subset' dataset if it exists
if 'person_subset' in datasets:
    fo.delete_dataset('person_subset')
# Delete 'dog_subset' dataset if it exists
if 'dog_subset' in datasets:
    fo.delete_dataset('dog_subset')
# Delete 'cat_subset' dataset if it exists
if 'cat_subset' in datasets:
    fo.delete_dataset('cat_subset')

# Load a split of a zoo dataset
person_subset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["classifications"],
    classes=["Person"],
    max_samples=subset_samples,
    seed=51,
    shuffle=True,
    dataset_name="person_subset",
)

cat_subset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["classifications"],
    classes=["Cat"],
    max_samples=subset_samples,
    seed=51,
    shuffle=True,
    dataset_name="cat_subset",
)

dog_subset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["classifications"],
    classes=["Dog"],
    max_samples=subset_samples,
    seed=51,
    shuffle=True,
    dataset_name="dog_subset",
)

Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading 'open-images-v7' split 'validation'
 100% |█████████████████| 250/250 [2.2s elapsed, 0s remaining, 120.5 samples/s]      
Dataset 'person_subset' created
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading 'open-images-v7' split 'validation'
 100% |█████████████████| 250/250 [2.5s elapsed, 0s remaining, 95.3 samples/s]       
Dataset 'cat_subset' created
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading 'open-images-v7' split 'validation'
 100% |█████████████████| 250/250 [1.7s elapsed, 0s remaining, 149.4 samples/s]         
Dataset 'dog_subset' created


In [225]:
# print out list of available datasets
print(fo.list_datasets())

['cat_subset', 'dog_subset', 'person_subset']


In [226]:
# load dataset
_dataset = fo.load_dataset("person_subset")
dog_subset = fo.load_dataset("dog_subset")
cat_subset = fo.load_dataset("cat_subset")
# Merge the samples together into the same dataset
_dataset.merge_samples(dog_subset)
_dataset.merge_samples(cat_subset)
print(_dataset.view())
fo.pprint(_dataset.stats(include_media=True))

Dataset:     person_subset
Media type:  image
Num samples: 675
Sample fields:
    id:              fiftyone.core.fields.ObjectIdField
    filepath:        fiftyone.core.fields.StringField
    tags:            fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    positive_labels: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classifications)
    negative_labels: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classifications)
View stages:
    ---
Computing metadata...
 100% |█████████████████| 675/675 [17.0s elapsed, 0s remaining, 163.6 samples/s]      
{
    'samples_count': 675,
    'samples_bytes': 764616,
    'samples_size': '746.7KB',
    'media_bytes': 201852636,
    'media_size': '192.5MB',
    'total_bytes': 202617252,
    'total_size': '193.2MB',
}


In [227]:
# Randomly shuffle the order of the samples in the dataset
_dataset = _dataset.shuffle()

In [228]:
sample_ids = _dataset.values("id")
# Split the sample IDs
train_ids, val_ids = train_test_split(sample_ids, test_size=0.3, random_state=42)
# Get the corresponding samples for training and testing
train_dataset = _dataset.select(train_ids)
val_dataset = _dataset.select(val_ids)
# Print the number of samples in each split
print("Number of training samples:", len(train_dataset))
print("Number of testing samples:", len(val_dataset))

Number of training samples: 472
Number of testing samples: 203


In [229]:
# Define model parametes
num_classes = 3

In [230]:
# Define the model
model_scratch = Sequential()
model_scratch.add(VGG19(include_top=False, weights=None, input_shape=(224, 224, 3)))  # VGG19 without the top layer
model_scratch.add(Flatten())
model_scratch.add(Dense(256, activation='relu'))
model_scratch.add(Dense(num_classes, activation='softmax'))

In [177]:
# Compile the model
model_scratch.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
model_scratch.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

NameError: name 'X_train' is not defined